In [ ]:
library(Seurat)
library(monocle)
library(dplyr)
library(RColorBrewer)
library(harmony)

In [ ]:
# load in raw data and Meta data from aligned data
Young.data <- read.csv("YAP5SA_DropSeq.raw.data.csv", sep=",")
Youngmeta.data <- read.csv("YAP5SA_DropSeq.meta.data.csv", sep=",")

In [ ]:
# Create Seurat object, only keeping cells that more than 10 reads and 500 genes
Young <- CreateSeuratObject(Young.data, meta.data = Youngmeta.data, min.cells = 10, min.features = 500)

In [ ]:
# Remove cells that contain more than 10000 RNA count and 
# that have more that 20 percent of reads that are mitochondrial
Young <- subset(Young, subset = nCount_RNA  <10000 & percent.mito < .2)

In [ ]:
# Normalizing, Finding Variable Features, Scaling Data and Find PCA of seurat Object
Young <- Young %>% 
  Seurat::NormalizeData(verbose = FALSE) %>%
  FindVariableFeatures(selection.method = "vst", nfeatures = 15000) %>% 
  ScaleData(verbose = FALSE) %>% 
  RunPCA(pc.genes = Young@var.genes, npcs = 20, verbose = FALSE)

In [ ]:
# Generating Harmony PCA from PCA subsequent PCA generated from normalized data
Young <- Young %>% 
  RunHarmony("orig.ident", plot_convergence = TRUE)

In [ ]:
#Elbowplot to visualize optimal harmony components to utilize for UMAP
ElbowPlot(Young, reduction = "harmony")

In [ ]:
# Dimensionally reduce harmony PCA (UMAP), 
# Construct KNN using harmony PCA
# Find Clusters at low granular level
Young <- Young %>% 
  RunUMAP(reduction = "harmony", dims = 1:20, min_dist=.001) %>% 
  FindNeighbors(reduction = "harmony", dims = 1:20) %>% 
  FindClusters(resolution = 0.1) %>% 
  identity()

In [ ]:
#Generate UMAP with resolution 0.1 as idents and save as PDF
pdf("Young_UMAP.pdf", width=10, height=10, useDingbats = FALSE)
DimPlot(Young,cols = brewer.pal(7,"Set3"), label=T) + 
theme_void() + 
theme(legend.position = "none") +
ggtitle("")
dev.off()

In [ ]:
#Find CellType Markers that are expressed in at least 50% of the cluster and a Log2FC of .5 or greater
Young_Markers <- FindAllMarkers(Young, only.pos=TRUE, min.pct=0.50, logfc.threshold=0.50)
Young_Markers <- subset(Young_Markers, subset=p_val_adj<0.05)
YoungTop5Markers <- Young_Markers %>% group_by(cluster) %>% top_n(n=5, wt=avg_log2FC)
YoungTop5MarkersUnique <- unique(YoungTop5Markers$gene)

In [ ]:
#Heatmap of top5 differentially expressed markers for each Cell type isolated
DoHeatmap(subset(Young, downsample = 100), features = YoungTop5MarkersUnique,  size = 3) + 
scale_fill_gradient2(low="white",mid="white",high="red")


In [ ]:
# Label Cell Clusters for appropriate Cell Type based on differentially expressed markers
Idents(Young) <- Young$`RNA_snn_res.0.1`
Young <- RenameIdents(Young, "0"="Mac")
Young <- RenameIdents(Young, "1"="CF")
Young <- RenameIdents(Young, "2"="CM")
Young <- RenameIdents(Young, "3"="Mural")
Young <- RenameIdents(Young, "4"="EC")
Young <- RenameIdents(Young, "5"='EpiC')
Young$CellType <- Idents(Young)

In [ ]:
#Generate UMAP with Celltype as idents and save as PDF
pdf("Young_UMAP_Celltyoe.pdf", width=10, height=10, useDingbats = FALSE)
DimPlot(Young,cols = brewer.pal(7,"Set3"), label=T) + 
theme_void() + 
theme(legend.position = "none") +
ggtitle("")
dev.off()

In [ ]:
library(dplyr)
#After Cluster labeling
#Find CellType Markers that are expressed in at least 50% of the cluster and a Log2FC of .5 or greater
Young_Markers <- FindAllMarkers(Young, only.pos=TRUE, min.pct=0.50, logfc.threshold=0.50)
Young_Markers <- subset(Young_Markers, subset=p_val_adj <.05)
YoungTop5Markers <- Young_Markers %>% group_by(cluster) %>% top_n(n=5, wt=avg_log2FC)
YoungTop5MarkersUnique <- unique(YoungTop5Markers$gene)
write.table(Young_Markers, "Young_Markers.txt", sep="\t")

In [ ]:
#After Cluster Labeling
#Heatmap of top5 differentially expressed markers for each Cell type isolated
DoHeatmap(subset(Young, downsample = 100), features = YoungTop5MarkersUnique,  size = 3) + 
scale_fill_gradient2(low="white",mid="white",high="grey")


In [ ]:
#Generate Heatmap for top 5 celltype markers
pdf("Young_Marker_heatmap.pdf", width=5, height=5)
DoHeatmap(subset(Young, downsample = 100), 
          group.colors = brewer.pal(11,"Set3"),
          features = YoungTop5MarkersUnique,  
          size = 3) + 
scale_fill_gradient2(low="white",mid="white",high="grey")
dev.off()

In [ ]:
#Extract numbers of celltype separated by Experiment
Counttable <- data.frame(table(Young$CellType, Young$Experiment))
write.csv(Counttable, "Young_Composition_table_Experiment.csv")
#Generate Cell Composition stack barplot
pdf("Young_Composition_Experiment.pdf", width=5, height=5)
ggplot(Counttable, aes(fill=Var1, y=Freq, x=Var2)) + 
    scale_fill_brewer(palette="Set3") + 
    geom_bar(position="fill", stat="identity") +
    theme_classic(base_size = 20) +
    xlab("Samples") + 
    ylab("Percentage") +   
    theme(legend.title=element_blank()) +
    RotatedAxis()
dev.off()

In [ ]:
#Extract numbers of celltype separated by Sample
Counttable <- data.frame(table(Young$CellType, Young$orig.ident))
write.csv(Counttable, "Young_Composition_table_Sample.csv")
#Generate Cell Composition stack barplot
pdf("Young_Composition.pdf", width=5, height=5)
ggplot(Counttable, aes(fill=Var1, y=Freq, x=Var2)) + 
    scale_fill_brewer(palette="Set3") + 
    geom_bar(position="fill", stat="identity") +
    theme_classic(base_size = 20) +
    xlab("Samples") + 
    ylab("Percentage") +   
    theme(legend.title=element_blank()) +
    RotatedAxis()
dev.off()

In [ ]:
#Subset Cardiomyocytes from "Young" object for Cell State Analysis
CMHarmony <- subset(Young, idents="CM")

In [ ]:
# Normalizing, Finding Variable Features, Scaling Data and Find PCA of Cardiomyocytes
CMHarmony <- CMHarmony %>% 
  FindVariableFeatures(selection.method = "vst", nfeatures = 15000) %>% 
  ScaleData(verbose = FALSE) %>% 
  RunPCA(pc.genes = Young@var.genes, npcs = 20, verbose = FALSE)


In [ ]:
# Generating Harmony PCA from PCA subsequent PCA generated from normalized data
CMHarmony <- CMHarmony %>% 
  RunHarmony("orig.ident", plot_convergence = TRUE)

In [ ]:
#Elbowplot to visualize optimal harmony components to utilize for UMAP
ElbowPlot(CMHarmony, reduction = "harmony")

In [ ]:
# Dimensionally reduce harmony PCA (UMAP), 
# Construct KNN using harmony PCA
# Find Clusters at low granular level
CMHarmony <- CMHarmony %>% 
  RunUMAP(reduction = "harmony", dims = 1:10) %>% 
  FindNeighbors(reduction = "harmony", dims = 1:10) %>% 
  FindClusters(resolution = 0.2) %>% 
  identity()

In [ ]:
#Dimplot of Cardimyocytes cell states at low resolution
DimPlot(CMHarmony, group.by="RNA_snn_res.0.1", label=T)

In [ ]:
#Find Cardiomyocyte cell state Markers that are expressed 
#in at least 50% of the cluster and a Log2FC of .5 or greater
library(dplyr)
Idents(CMHarmony) <- CMHarmony$`RNA_snn_res.0.1`
CMHarmony_Markers <- FindAllMarkers(CMHarmony, only.pos=TRUE, min.pct=0.25, logfc.threshold=0.25)
CMHarmony_Markers <- subset(CMHarmony_Markers, subset=p_val_adj <.05)
CMHarmony_Markers <- CMHarmony_Markers[!grepl("mt-", CMHarmony_Markers$gene),]
CMHarmony_Markers <- CMHarmony_Markers[!grepl("-", CMHarmony_Markers$gene),]
CMHarmony_Markers <- CMHarmony_Markers[!grepl("Gm", CMHarmony_Markers$gene),]
CMHarmonyTop5Markers <- CMHarmony_Markers %>% group_by(cluster) %>% top_n(n=5, wt=avg_log2FC)
CMHarmonyTop5MarkersUnique <- unique(CMHarmonyTop5Markers$gene)

In [ ]:
#Heatmap of top5 differentially expressed markers for each CM Cell state isolated
DoHeatmap(subset(CMHarmony, downsample = 100), features = CMHarmonyTop5MarkersUnique,  size = 3) + 
scale_fill_gradient2(low="white",mid="white",high="grey")

In [ ]:
#Labeling of CM Cell states 
Idents(CMHarmony) <- CMHarmony$`RNA_snn_res.0.1`
CMHarmony <- RenameIdents(CMHarmony, "2"="CM A")
CMHarmony <- RenameIdents(CMHarmony, "0"="CM A")
CMHarmony <- RenameIdents(CMHarmony, "1"="CM B")
CMHarmony <- RenameIdents(CMHarmony, "4"="CM C")
CMHarmony <- RenameIdents(CMHarmony, "5"="CM D")
CMHarmony <- RenameIdents(CMHarmony, "3"="CM G2M")
CMHarmony$Idents <- Idents(CMHarmony)
CMHarmony$Idents <- factor(CMHarmony$Idents, levels=c("CM A","CM B","CM C","CM D","CM G2M"))

In [ ]:
#Generate UMAP of Cardiomyocyte cell states
DimPlot(CMHarmony, label=T,label.size = 10) +
theme(plot.title = element_text(hjust = 0.5,face = "bold")) +
theme(legend.position = "none") +
ggtitle("Cardiomyocyte Cell State") +
theme(
  axis.text.x = element_blank(),
  axis.text.y = element_blank(),
axis.title.x = element_text(face="bold"),
axis.title.y = element_text(face="bold"),
  axis.ticks = element_blank(),
  axis.line.y = element_line(size = 1.2),
  axis.line.x = element_line(size = 1.2)) +
xlab("UMAP1") +
ylab("UMAP2")

In [ ]:
#Find DE Markers for each Cell state
library(dplyr)
Idents(CMHarmony) <- CMHarmony$Idents
CMHarmony_Markers <- FindAllMarkers(CMHarmony, only.pos=TRUE, min.pct=0.25, logfc.threshold=0.25)
CMHarmony_Markers <- subset(CMHarmony_Markers, subset=p_val_adj <.05)
CMHarmony_Markers <- CMHarmony_Markers[!grepl("mt-", CMHarmony_Markers$gene),]
CMHarmony_Markers <- CMHarmony_Markers[!grepl("-", CMHarmony_Markers$gene),]
CMHarmony_Markers <- CMHarmony_Markers[!grepl("Gm", CMHarmony_Markers$gene),]
write.table(CMHarmony_Markers, "CMHarmony_Markers.txt", sep="\t")
CMHarmonyTop5Markers <- CMHarmony_Markers %>% group_by(cluster) %>% top_n(n=5, wt=avg_log2FC)
CMHarmonyTop5MarkersUnique <- unique(CMHarmonyTop5Markers$gene)

In [ ]:
#Generate Heatmap of Cardiomyocyte Cell State Markers
pdf("CMHarmony_Marker_heatmap.pdf", width=5, height=5)
DoHeatmap(subset(CMHarmony, downsample = 100), features = CMHarmonyTop5MarkersUnique,  size = 3) + 
scale_fill_gradient2(low="white",mid="white",high="grey")
dev.off()

In [ ]:
#Extract Cell Count of Cardiomyocytes by Cell State and Experimental Design
Counttable <- data.frame(table(CMHarmony$Idents, CMHarmony$Experiment))
write.csv(Counttable, "CMHarmony_Composition_Experiment_table.csv")
#Generate Cell Composition Stack Barplot of Cardiomyocyte cell state
pdf("CMHarmony_Composition_Experiment.pdf", width=6, height=6)
ggplot(Counttable, aes(fill=Var1, y=Freq, x=Var2)) + 
    scale_fill_brewer(palette="Set3") + 
    geom_bar(position="fill", stat="identity") +
    theme_classic(base_size = 20) +
    xlab("Samples") + 
    ylab("Percentage") +   
    theme(legend.title=element_blank()) +
    RotatedAxis()
dev.off()

In [ ]:
#Extract Cell Count of Cardiomyocytes by Cell State and Sample
Counttable <- data.frame(table(CMHarmony$Idents, CMHarmony$orig.ident))
write.csv(Counttable, "CMHarmony_Composition_Sample_table.csv")
#Generate Cell Composition Stack Barplot of Cardiomyocyte cell state
pdf("CMHarmony_Composition_Sample.pdf", width=6, height=6)
ggplot(Counttable, aes(fill=Var1, y=Freq, x=Var2)) + 
    scale_fill_brewer(palette="Set3") + 
    geom_bar(position="fill", stat="identity") +
    theme_classic(base_size = 20) +
    xlab("Samples") + 
    ylab("Percentage") +   
    theme(legend.title=element_blank()) +
    RotatedAxis()
dev.off()

In [ ]:
#Extract Cell Count of Cardiomyocytes B & G2M by Cell State and Sample
Counttable <- data.frame(table(CMHarmony2$Idents, CMHarmony2$orig.ident))
#Generate Cell Composition Stack Barplot of Cardiomyocyte cell state
pdf("CMHarmony2_Composition_Sample.pdf", width=6, height=6)
ggplot(Counttable, aes(fill=Var1, y=Freq, x=Var2)) + 
    scale_fill_brewer(palette="Set3") + 
    geom_bar(position="fill", stat="identity") +
    theme_classic(base_size = 20) +
    xlab("Samples") + 
    ylab("Percentage") +   
    theme(legend.title=element_blank()) +
    RotatedAxis()
dev.off()

In [ ]:
#Subset CM B and CM G2M for further analysis
DefaultAssay(CMHarmony) <- "RNA"
CMHarmony2 <- subset(CMHarmony, idents=c("CM B","CM G2M"))

In [ ]:
#Rescale and find PCA for CM B and CM G2M (CMHarmony2)
CMHarmony2 <- CMHarmony2 %>%
  FindVariableFeatures(selection.method = "vst", nfeatures = 15000) %>% 
  ScaleData(verbose = FALSE) %>% 
  RunPCA(pc.genes = Young@var.genes, npcs = 20, verbose = FALSE)

In [ ]:
# Generating Harmony PCA from PCA subsequent PCA generated from normalized data
CMHarmony2 <- CMHarmony2 %>% 
  RunHarmony("orig.ident", plot_convergence = TRUE)

In [ ]:
ElboePlot(CMHarmony2, reduction="harmony")

In [ ]:
# Dimensionally reduce harmony PCA (UMAP), 
# Construct KNN using harmony PCA
# Find Clusters at low granular level
CMHarmony2 <- CMHarmony2 %>% 
  RunUMAP(reduction = "harmony", dims = 1:10) %>% 
  FindNeighbors(reduction = "harmony", dims = 1:10) %>% 
  FindClusters(resolution = .1) %>% 
  identity()

In [ ]:
#Set Identity and reorder
CMHarmony2$Idents <- Idents(CMHarmony2)
CMHarmony2$Idents <- factor(CMHarmony2$Idents, level=c("1","2","3","4","5"))

In [ ]:
#Find Cardiomyocyte cell state Markers that are expressed 
#in at least 50% of the cluster and a Log2FC of .5 or greater
library(dplyr)
Idents(CMHarmony2) <- CMHarmony2$Idents
CMHarmony2_Markers <- FindAllMarkers(CMHarmony2, only.pos=TRUE, min.pct=0.25, logfc.threshold=0.5)
CMHarmony2_Markers <- subset(CMHarmony2_Markers, subset=p_val_adj <.05)
CMHarmony2_Markers <- CMHarmony2_Markers[!grepl("mt-", CMHarmony2_Markers$gene),]
CMHarmony2_Markers <- CMHarmony2_Markers[!grepl("-", CMHarmony2_Markers$gene),]
CMHarmony2_Markers <- CMHarmony2_Markers[!grepl("Rik", CMHarmony2_Markers$gene),]
CMHarmony2Top5Markers <- CMHarmony2_Markers %>% group_by(cluster) %>% top_n(n=5, wt=avg_log2FC)
CMHarmony2Top5MarkersUnique <- unique(CMHarmony2Top5Markers$gene)
write.table(CMHarmony2_Markers, "CMHarmony2_Markers.txt", sep="\t")

In [ ]:
#Heatmap of top5 differentially expressed markers for each CM B & G2M Substates
DoHeatmap(subset(CMHarmony2, downsample = 100), features = CMHarmony2Top5MarkersUnique,  size = 3) + 
scale_fill_gradient2(low="white",mid="white",high="grey")

# Round Score Analysis

In [ ]:
#Read in Rounding Target List
RoundingTargets <- read.table("RoundingTargets.txt", sep="\t")

In [ ]:
#Add module Score to Seurat CM Objects using Rounding Target list
DefaultAssay(CMHarmony) <- "RNA"
CMHarmony <- AddModuleScore(CMHarmony, features = list(RoundingTargets$V1), name = "Rounding_Score")

DefaultAssay(CMHarmony) <- "RNA"
CMHarmony2 <- AddModuleScore(CMHarmony2, features = list(RoundingTargets$V1), name = "Rounding_Score")

In [ ]:
#UMAP visualization of Rounding score on CM B & G2M substate
FeaturePlot(CMHarmony, 
            features = "Rounding_Score1", min.cutoff="q9") + 
theme(plot.title = element_text(hjust = 0.5,face = "bold")) +
theme(legend.position = "right") +
ggtitle("Rounding Score") +
theme(
  axis.text.x = element_blank(),
  axis.text.y = element_blank(),
axis.title.x = element_text(face="bold"),
axis.title.y = element_text(face="bold"),
  axis.ticks = element_blank(),
  axis.line.y = element_line(size = 1),
  axis.line.x = element_line(size = 1)) +
xlab("UMAP1") +
ylab("UMAP2") + scale_colour_gradientn(colours = brewer.pal(n = 11, name = "YlOrRd"))

In [ ]:
#UMAP visualization of Rounding score on CM B & G2M substate
FeaturePlot(CMHarmony2, 
            features = "Rounding_Score1", min.cutoff="q9") + 
theme(plot.title = element_text(hjust = 0.5,face = "bold")) +
theme(legend.position = "right") +
ggtitle("Rounding Score") +
theme(
  axis.text.x = element_blank(),
  axis.text.y = element_blank(),
axis.title.x = element_text(face="bold"),
axis.title.y = element_text(face="bold"),
  axis.ticks = element_blank(),
  axis.line.y = element_line(size = 1),
  axis.line.x = element_line(size = 1)) +
xlab("UMAP1") +
ylab("UMAP2") + scale_colour_gradientn(colours = brewer.pal(n = 11, name = "YlOrRd"))

# P2 Scoring

In [ ]:
#Read in genelist for Differentially expressed gene between 4 weeks vs P2 Control Cadiomyocytes
#Add modular scoring to CM object using genes that are upregulated in P2 vs weeks
CMControlDEG <- read.table("WithOlsenSingleCell/CMControlDEG.txt", sep="\t")
CMControlDEGDown <- subset(CMControlDEG, subset=avg_log2FC < 0)
CMHarmony <- AddModuleScore(CMHarmony, features = list(rownames(CMControlDEGDown)),  name="P2_Score")

In [ ]:
#UMAP visualization of P2 score on CM B & G2M substate
FeaturePlot(CMHarmony, 
            features = "P2_Score1", min.cutoff="q9") +
theme(plot.title = element_text(hjust = 0.5,face = "bold")) +
theme(legend.position = "right") +
ggtitle("Rounding Score") +
theme(
  axis.text.x = element_blank(),
  axis.text.y = element_blank(),
axis.title.x = element_text(face="bold"),
axis.title.y = element_text(face="bold"),
  axis.ticks = element_blank(),
  axis.line.y = element_line(size = 1),
  axis.line.x = element_line(size = 1)) +
xlab("UMAP1") +
ylab("UMAP2") + scale_colour_gradientn(colours = brewer.pal(n = 11, name = "YlOrRd"))

# Monocle2 Analysis

In [ ]:
DefaultAssay(CMHarmony2) <- 'RNA'

In [ ]:
#Extract Gene Matrix, Gene Names, and meta data from CM B & G2M substates
Data <- as.matrix(GetAssayData(CMHarmony2))
Genes <- data.frame(rownames(CMHarmony2))
rownames(Genes) <- Genes[,1]
colnames(Genes)[1] <- "gene_short_name"
Meta <- data.frame(CMHarmony2@meta.data)

In [ ]:
#Create Monocle Object
CMHarmony2Monocle <- newCellDataSet(as.matrix(Data), 
                          featureData = new("AnnotatedDataFrame", data=Genes), 
                          phenoData = new("AnnotatedDataFrame", data=Meta))

In [ ]:
#Find Size Factor, Ordering Filters, and Reduce Dimensions using DDTree
#Order Cells along trajectory
CMHarmony2Monocle <- estimateSizeFactors(CMHarmony2Monocle)
CMHarmony2Monocle <- setOrderingFilter(Monocle, CMHarmony2_Markers$gene)
CMHarmony2Monocle <- reduceDimension(Monocle, max_components = 2, method="DDRTree")
CMHarmony2Monocle <- orderCells(Monocle)

In [ ]:
MonocleName <- c("CMHarmony2")
MonocleList <- list("CMHarmony2"=Monocle)
for(i in MonocleName)
{
  jpeg(paste(i,"_Cell_Trajectory_State.jpg",sep=""),width=1000,height=700,res=200)
  g <- plot_cell_trajectory(MonocleList[[i]],
                            color_by="State",
                            show_state_number = F, 
                            show_branch_points = FALSE) + 
    scale_color_brewer(palette = "Set3")
  print(g)
  dev.off()
}

for(i in MonocleName)
{
  jpeg(paste(i,"_Cell_Trajectory_Experiment.jpg",sep=""),width=1000,height=500,res=200)
  g <- plot_cell_trajectory(MonocleList[[i]],
                            color_by="Experiment",
                            show_branch_points = FALSE) + 
    theme_void()
  print(g)
  dev.off()
}

for(i in MonocleName)
{
  jpeg(paste(i,"_Cell_Trajectory_Idents.jpg",sep=""),width=1000,height=500,res=200)
  g <- plot_cell_trajectory(MonocleList[[i]], 
                            color_by="Idents2",
                            show_branch_points = TRUE) + 
    theme_void()
  print(g)
  dev.off()
}
for(i in MonocleName)
{
  jpeg(paste(i,"_Cell_Trajectory_Idents_split.jpg",sep=""),width=4000,height=500,res=200)
  g <- plot_cell_trajectory(MonocleList[[i]],
                            color_by="Idents2") + 
    facet_wrap(~Idents2, nrow=1) + theme_void()
  print(g)
  dev.off()
}
for(i in MonocleName)
{
  jpeg(paste(i,"_Cell_Trajectory_Pseudotime.jpg",sep=""),width=1000,height=500,res=200)
  g <- plot_cell_trajectory(MonocleList[[i]],
                            color_by="Pseudotime",
                            show_branch_points = TRUE) + 
    theme_void()
  print(g)
  dev.off()
}



dev.off()

Matrix <- data.frame(t(MonocleList[[i]]@reducedDimS))
Idents <- data.frame(MonocleList[[i]]$Idents)
Idents <- str(Idents)
Matrix <- cbind(Matrix,Idents)
colnames(Matrix)[1] <- "Comp1"
colnames(Matrix)[2] <- "Comp2"
colnames(Matrix)[3] <- "Cluster"

for(i in MonocleName)
{
  Matrix <- data.frame(t(MonocleList[[i]]@reducedDimS))
  CFIdents <- data.frame(MonocleList[[i]]$Idents)
  Matrix <- cbind(Matrix,CFIdents)
  colnames(Matrix)[1] <- "Comp1"
  colnames(Matrix)[2] <- "Comp2"
  colnames(Matrix)[3] <- "Cluster"
  jpeg(paste(i,"_RidgePlot.jpg",sep=""), width=1080, res=200)
  g <- ggplot(Matrix, aes(x=Comp1, y=Cluster)) + 
    geom_density_ridges(aes(fill=Cluster)) + 
    theme_classic()
  print(g)
  dev.off()
  rm(Matrix)
  rm(CFIdents)
}

for(i in MonocleName)
{
  Matrix <- data.frame(t(MonocleList[[i]]@reducedDimS))
  CFIdents <- data.frame(MonocleList[[i]]$Idents)
  Matrix <- cbind(Matrix,CFIdents)
  colnames(Matrix)[1] <- "Comp1"
  colnames(Matrix)[2] <- "Comp2"
  colnames(Matrix)[3] <- "Cluster"
  jpeg(paste(i,"_RidgePlot2.jpg",sep=""), width=1080, res=200)
  g <- ggplot(Matrix, aes(x=Comp2, y=Cluster)) + 
    geom_density_ridges(aes(fill=Cluster)) + 
    theme_classic()
  print(g)
  dev.off()
  rm(Matrix)
  rm(CFIdents)
}

for(i in MonocleName)
{
  Matrix <- data.frame(t(MonocleList[[i]]@reducedDimS))
  CFIdents <- data.frame(MonocleList[[i]]$Experiment)
  Matrix <- cbind(Matrix,CFIdents)
  colnames(Matrix)[1] <- "Comp1"
  colnames(Matrix)[2] <- "Comp2"
  colnames(Matrix)[3] <- "Cluster"
  jpeg(paste(i,"_RidgePlot_Experiment.jpg",sep=""), width=1080, res=200)
  g <- ggplot(Matrix, aes(x=Comp1, y=Cluster)) + 
    geom_density_ridges(aes(fill=Cluster)) + 
    theme_classic()
  print(g)
  dev.off()
  rm(Matrix)
  rm(CFIdents)
}

for(i in MonocleName)
{
  Matrix <- data.frame(t(MonocleList[[i]]@reducedDimS))
  CFIdents <- data.frame(MonocleList[[i]]$Experiment)
  Matrix <- cbind(Matrix,CFIdents)
  colnames(Matrix)[1] <- "Comp1"
  colnames(Matrix)[2] <- "Comp2"
  colnames(Matrix)[3] <- "Cluster"
  jpeg(paste(i,"_RidgePlot_Experiment2.jpg",sep=""), width=1080, res=200)
  g <- ggplot(Matrix, aes(x=Comp2, y=Cluster)) + 
    geom_density_ridges(aes(fill=Cluster)) + 
    theme_classic()
  print(g)
  dev.off()
  rm(Matrix)
  rm(CFIdents)
}
gc()